<a href="https://colab.research.google.com/github/BhavyaaaD/BlogPreviewPost/blob/main/DynamicAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf



In [ ]:
df=pd.read_csv('/content/feature_vectors_syscalls_frequency_5_Cat.csv')
df.head()

,__arm_nr_cacheflush,__arm_nr_set_tls,_llseek,_newselect,accept,access,bind,brk,capset,chdir,...,ugetrlimit,umask,uname,unlink,utimes,vfork,wait4,write,writev,Class
0,0.0,14.0,6.0,0.0,0.0,11.0,3.0,21.0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,37.0,10.0,1.0
1,1590.0,42.0,6.0,0.0,0.0,47.0,2.0,1430.0,1.0,0.0,...,2.0,0.0,1.0,4.0,0.0,0.0,0.0,2840.0,46.0,1.0
2,0.0,23.0,6.0,0.0,0.0,40.0,9.0,89.0,1.0,0.0,...,0.0,0.0,1.0,2.0,0.0,0.0,0.0,111.0,20.0,1.0
3,0.0,27.0,6.0,0.0,0.0,60.0,1.0,242.0,1.0,0.0,...,0.0,0.0,1.0,2.0,0.0,0.0,0.0,987.0,197.0,1.0
4,0.0,18.0,6.0,0.0,0.0,26.0,5.0,59.0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,98.0,25.0,1.0


In [ ]:
cols=df.select_dtypes(include='float').columns


In [ ]:
for i in cols:
  df[i]=df[i].astype(int)

In [ ]:
df_x=df.iloc[:,0:139]
df_x.head()

,__arm_nr_cacheflush,__arm_nr_set_tls,_llseek,_newselect,accept,access,bind,brk,capset,chdir,...,truncate,ugetrlimit,umask,uname,unlink,utimes,vfork,wait4,write,writev
0,0,14,6,0,0,11,3,21,1,0,...,0,0,0,1,0,0,0,0,37,10
1,1590,42,6,0,0,47,2,1430,1,0,...,0,2,0,1,4,0,0,0,2840,46
2,0,23,6,0,0,40,9,89,1,0,...,0,0,0,1,2,0,0,0,111,20
3,0,27,6,0,0,60,1,242,1,0,...,0,0,0,1,2,0,0,0,987,197
4,0,18,6,0,0,26,5,59,1,0,...,0,0,0,1,0,0,0,0,98,25


In [ ]:
df_y=df['Class']
df_y=df_y.replace({5:0,4:1,3:1,2:1})


In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test = train_test_split(df_x,df_y)

In [ ]:
X_train.shape


(8698, 139)

In [ ]:
X_test.shape

(2900, 139)

In [ ]:
x_train=X_train.values
y_train=Y_train.values
x_test=X_test.values
y_test=Y_test.values

In [ ]:
x_train=x_train[...,tf.newaxis].astype('float32')
x_test=x_test[...,tf.newaxis].astype('float32')

In [ ]:
x_train.shape

(8698, 139, 1)

In [ ]:
y_test.shape

(2900,)

In [ ]:
y_train=np.array(pd.get_dummies(y_train))
y_test=np.array(pd.get_dummies(y_test))

In [ ]:
y_test

array([[1, 0],
       [0, 1],
       [0, 1],
       ...,
       [1, 0],
       [0, 1],
       [0, 1]], dtype=uint8)

In [ ]:
from tensorflow.keras.layers import Dense,Flatten,Bidirectional,GRU,LSTM
from tensorflow.keras.models import Sequential

In [ ]:
model=Sequential([
    LSTM(128,input_shape=(139,1)),

    Dense(64,activation='sigmoid'),
    Dense(32,activation='sigmoid'),

    Dense(2,activation='softmax')

])

In [ ]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_1 (LSTM)               (None, 128)               66560     
                                                                 
 dense_3 (Dense)             (None, 64)                8256      
                                                                 
 dense_4 (Dense)             (None, 32)                2080      
                                                                 
 dense_5 (Dense)             (None, 2)                 66        
                                                                 
Total params: 76,962
Trainable params: 76,962
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.compile(optimizer='adam',loss=tf.keras.losses.categorical_crossentropy,metrics=['accuracy'])

In [ ]:
model.fit(x_train,y_train,epochs=10,batch_size=32)

Epoch 1/10
272/272 [==============================] - 42s 155ms/step - loss: 0.2460 - accuracy: 0.9081
Epoch 2/10
272/272 [==============================] - 41s 152ms/step - loss: 0.2301 - accuracy: 0.9152
Epoch 3/10
272/272 [==============================] - 43s 157ms/step - loss: 0.2239 - accuracy: 0.9170
Epoch 4/10
272/272 [==============================] - 41s 149ms/step - loss: 0.2105 - accuracy: 0.9215
Epoch 5/10
272/272 [==============================] - 40s 149ms/step - loss: 0.2040 - accuracy: 0.9209
Epoch 6/10
272/272 [==============================] - 41s 150ms/step - loss: 0.1977 - accuracy: 0.9268
Epoch 7/10
272/272 [==============================] - 41s 150ms/step - loss: 0.1864 - accuracy: 0.9318
Epoch 8/10
272/272 [==============================] - 40s 149ms/step - loss: 0.1768 - accuracy: 0.9352
Epoch 9/10
272/272 [==============================] - 42s 153ms/step - loss: 0.1711 - accuracy: 0.9395
Epoch 10/10
272/272 [==============================] - 40s 148ms/step - l

In [ ]:
model.evaluate(x_test,y_test)

91/91 [==============================] - 5s 50ms/step - loss: 0.1864 - accuracy: 0.9366


[0.1863517165184021, 0.9365517497062683]